    # What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary.

Given that our stakeholders are seeking to understand what factors drive the price of a used car then their goal is to improve their sales and meet market demand. They have provided as a dataset of used car sales and have asked us to determine which factors make a car more or less expensive. A recent [report](https://www.statista.com/statistics/183713/value-of-us-passenger-cas-sales-and-leases-since-1990/) shows that used car sales doubles the amount of new car sales. The used car market is lucrative with high demand from consumers. Our stakeholder wants to stay ahead of the curve and understanding their customer's preferences can help lead to more sales. Used car dealerships also have to acquire cars for inventory from other entities. If our stakeholder can reasonably determine the cost of a used vehicle they can better understand their margins and when to purchase a vehicle for inventory. It is also important to understand what customers value in used cars. Knowing what drives a consumer to purchase a used vehicle can help our stakeholders avoid costly mistakes when acquiring inventory.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio

from plotly.subplots import make_subplots
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load data
vehicles_df = pd.read_csv('data/vehicles.csv')

In [4]:
# First we should get the basic facts of the data (describe, info, null checks, duplicate checks)
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

The id and VIN column are not important for analysis so we will definitely remove them.

In [5]:
vehicles_df.describe()

,id,price,year,odometer
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07


In [6]:
vehicles_df.describe(include='object')

,region,manufacturer,model,condition,cylinders,fuel,title_status,transmission,VIN,drive,size,type,paint_color,state
count,426880,409234,421603,252776,249202,423867,418638,424324,265838,296313,120519,334022,296677,426880
unique,404,42,29649,6,8,5,6,3,118246,3,4,13,12,51
top,columbus,ford,f-150,good,6 cylinders,gas,clean,automatic,1FMJU1JT1HEA52352,4wd,full-size,sedan,white,ca
freq,3608,70985,8009,121456,94169,356209,405117,336524,261,131904,63465,87056,79285,50614


In [10]:
print(f'Row count: {vehicles_df.shape[0]}, Duplicate count: {vehicles_df.shape[0] - vehicles_df.drop_duplicates().shape[0]}')

Row count: 426880, Duplicate count: 0


In [7]:
vehicles_df.isna().mean().round(2)

id              0.00
region          0.00
price           0.00
year            0.00
manufacturer    0.04
model           0.01
condition       0.41
cylinders       0.42
fuel            0.01
odometer        0.01
title_status    0.02
transmission    0.01
VIN             0.38
drive           0.31
size            0.72
type            0.22
paint_color     0.31
state           0.00
dtype: float64

Luckily, we do not have duplicates. However, we are missing values, specifically the condition and cylinders attributes are missing in over 40% of rows. Color is missing in 31% of rows. Size is missing in 72% of rows. Drive is missing in 31% of rows. I know that these attributes are important in determining the price of a car from prior analysis. The amount of missing data is concerning and the best course of action may be to impute them. That may mislead our model and we should be wary of that. However, losing over 40% of the data isn't something we should do lightly. One way to impute this data would be to find a finding an identical year and model then we would select the most seen value for that respective column. This can help us deduce the missing attributes.

I will create an imputed dataset and run my analysis on both of them.

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.